# Tools

Tools are code that can be executed by an agent to perform actions. A tool
can be a simple function such as a calculator, or an API call to a third-party service
such as stock price lookup and weather forecast.
In the context of AI agents, tools are designed to be executed by agents in
response to model-generated function calls.

AGNext provides the {py:mod}`agnext.components.tools` module with a suite of built-in
tools and utilities for creating and running custom tools.

## Built-in Tools

One of the built-in tools is the {py:class}`agnext.components.tools.PythonCodeExecutionTool`,
which allows agents to execute Python code snippets.

Here is how you create the tool and use it.

In [ ]:
from agnext.components.code_executor import LocalCommandLineCodeExecutor
from agnext.components.tools import PythonCodeExecutionTool
from agnext.core import CancellationToken

# Create the tool.
code_executor = LocalCommandLineCodeExecutor()
code_execution_tool = PythonCodeExecutionTool(code_executor)
cancellation_token = CancellationToken()

# Use the tool directly without an agent.
code = "print('Hello, world!')"
result = await code_execution_tool.run_json({"code": code}, cancellation_token)
print(code_execution_tool.return_value_as_string(result))

The {py:class}`~agnext.components.code_executor.LocalCommandLineCodeExecutor`
class is a built-in code executor that runs Python code snippets in a subprocess
in the local command line environment.
The {py:class}`~agnext.components.tools.PythonCodeExecutionTool` class wraps the code executor
and provides a simple interface to execute Python code snippets.

Other built-in tools will be added in the future.

## Custom Function Tools

A tool can also be a simple Python function that performs a specific action.
To create a custom function tool, you just need to create a Python function
and use the {py:class}`agnext.components.tools.FunctionTool` class to wrap it.

For example, a simple tool to obtain the stock price of a company might look like this:

In [5]:
import random

from agnext.components.tools import FunctionTool
from agnext.core import CancellationToken
from typing_extensions import Annotated


async def get_stock_price(ticker: str, date: Annotated[str, "Date in YYYY/MM/DD"]) -> float:
    # Returns a random stock price for demonstration purposes.
    return random.uniform(10, 200)


# Create a function tool.
stock_price_tool = FunctionTool(get_stock_price, description="Get the stock price.")

# Run the tool.
cancellation_token = CancellationToken()
result = await stock_price_tool.run_json({"ticker": "AAPL", "date": "2021/01/01"}, cancellation_token)

# Print the result.
print(stock_price_tool.return_value_as_string(result))

138.75280591295171


## Tool-Equipped Agent

To use tools with an agent, you can use {py:class}`agnext.components.tool_agent.ToolAgent`,
by using it in a composition pattern.
Here is an example tool-use agent that uses {py:class}`~agnext.components.tool_agent.ToolAgent`
as an inner agent for executing tools.

In [6]:
import asyncio
from dataclasses import dataclass
from typing import List

from agnext.application import SingleThreadedAgentRuntime
from agnext.components import FunctionCall, TypeRoutedAgent, message_handler
from agnext.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    FunctionExecutionResult,
    FunctionExecutionResultMessage,
    LLMMessage,
    OpenAIChatCompletionClient,
    SystemMessage,
    UserMessage,
)
from agnext.components.tool_agent import ToolAgent, ToolException
from agnext.components.tools import FunctionTool, Tool, ToolSchema
from agnext.core import AgentId, CancellationToken


@dataclass
class Message:
    content: str


class ToolUseAgent(TypeRoutedAgent):
    def __init__(self, model_client: ChatCompletionClient, tool_schema: List[ToolSchema], tool_agent: AgentId) -> None:
        super().__init__("An agent with tools")
        self._system_messages: List[LLMMessage] = [SystemMessage("You are a helpful AI assistant.")]
        self._model_client = model_client
        self._tool_schema = tool_schema
        self._tool_agent = tool_agent

    @message_handler
    async def handle_user_message(self, message: Message, cancellation_token: CancellationToken) -> Message:
        # Create a session of messages.
        session: List[LLMMessage] = [UserMessage(content=message.content, source="user")]
        # Get a response from the model.
        response = await self._model_client.create(
            self._system_messages + session, tools=self._tool_schema, cancellation_token=cancellation_token
        )
        # Add the response to the session.
        session.append(AssistantMessage(content=response.content, source="assistant"))

        # Keep iterating until the model stops generating tool calls.
        while isinstance(response.content, list) and all(isinstance(item, FunctionCall) for item in response.content):
            # Execute functions called by the model by sending messages to itself.
            results: List[FunctionExecutionResult | BaseException] = await asyncio.gather(
                *[self.send_message(call, self._tool_agent) for call in response.content],
                return_exceptions=True,
            )
            # Combine the results into a single response and handle exceptions.
            function_results: List[FunctionExecutionResult] = []
            for result in results:
                if isinstance(result, FunctionExecutionResult):
                    function_results.append(result)
                elif isinstance(result, ToolException):
                    function_results.append(FunctionExecutionResult(content=f"Error: {result}", call_id=result.call_id))
                elif isinstance(result, BaseException):
                    raise result  # Unexpected exception.
            session.append(FunctionExecutionResultMessage(content=function_results))
            # Query the model again with the new response.
            response = await self._model_client.create(
                self._system_messages + session, tools=self._tool_schema, cancellation_token=cancellation_token
            )
            session.append(AssistantMessage(content=response.content, source=self.metadata["type"]))

        # Return the final response.
        assert isinstance(response.content, str)
        return Message(content=response.content)

The `ToolUseAgent` class is a bit involved, however,
the core idea can be described using a simple control flow graph:

![ToolUseAgent control flow graph](tool-use-agent-cfg.svg)

The `ToolUseAgent`'s `handle_user_message` handler handles messages from the user,
and determines whether the model has generated a tool call.
If the model has generated tool calls, then the handler sends a function call
message to the {py:class}`~agnext.components.tool_agent.ToolAgent` agent
to execute the tools,
and then queries the model again with the results of the tool calls.
This process continues until the model stops generating tool calls,
at which point the final response is returned to the user.

By having the tool execution logic in a separate agent,
we expose the model-tool interactions to the agent runtime as messages, so the tool executions
can be observed externally and intercepted if necessary.

To run the agent, we need to create a runtime and register the agent.

In [7]:
# Create a runtime.
runtime = SingleThreadedAgentRuntime()
# Create the tools.
tools: List[Tool] = [FunctionTool(get_stock_price, description="Get the stock price.")]
# Register the agents.
tool_executor_agent = await runtime.register_and_get(
    "tool-executor-agent",
    lambda: ToolAgent(
        description="Tool Executor Agent",
        tools=tools,
    ),
)
tool_use_agent = await runtime.register_and_get(
    "tool-use-agent",
    lambda: ToolUseAgent(
        OpenAIChatCompletionClient(model="gpt-4o-mini"),
        tool_schema=[tool.schema for tool in tools],
        tool_agent=tool_executor_agent,
    ),
)

This example uses the {py:class}`agnext.components.models.OpenAIChatCompletionClient`,
for Azure OpenAI and other clients, see [Model Clients](./model-clients.ipynb).
Let's test the agent with a question about stock price.

In [8]:
# Start processing messages.
run_context = runtime.start()
# Send a direct message to the tool agent.
response = await runtime.send_message(Message("What is the stock price of NVDA on 2024/06/01?"), tool_use_agent)
print(response.content)
# Stop processing messages.
await run_context.stop()

The stock price of NVDA on June 1, 2024, is approximately $49.28.


See [samples](https://github.com/microsoft/agnext/tree/main/python/samples#tool-use-examples)
for more examples of using tools with agents, including how to use
broadcast communication model for tool execution, and how to intercept tool
execution for human-in-the-loop approval.